<a href="https://colab.research.google.com/github/ScriptSherpa/Web-scrapping/blob/main/web_scrapping_iv_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

indeed job scrapper

In [6]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

 setup the query and url

In [3]:

def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [4]:
url = get_url('senior accountant', 'charlotte nc')
print(url)

https://www.indeed.com/jobs?q=senior+accountant&l=charlotte+nc


Extract the html data

In [9]:
response =requests.get(url)

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

In [11]:
cards = soup.find_all('div', 'jobsearch-SerpJobCard')

Prototype the model with a single record

In [15]:
card = cards[0]

IndexError: list index out of range

In [17]:
import requests
from bs4 import BeautifulSoup

# Get the URL
url = get_url('senior accountant', 'charlotte nc')
print(url)

# Fetch the HTML
response = requests.get(url)

# Check response status code
if response.status_code != 200:
    print(f"Request failed with status code: {response.status_code}")
else:
    # Parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all job cards
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')

    # Print the number of cards found
    print(f"Found {len(cards)} job cards.")

    # If cards are found, then access the first card
    if cards:
        card = cards[0]
    else:
        print("No job cards found. Please check the CSS selector or the Indeed website structure.")

https://www.indeed.com/jobs?q=senior+accountant&l=charlotte+nc
Request failed with status code: 403


In [16]:
job_title = card.h2.a.get('title')

NameError: name 'card' is not defined

In [ ]:
company = card.find('span', 'company').text.strip()
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')

In [ ]:
post_date = card.find('span', 'date').text
today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
summary = card.find('div', 'summary').text.strip().replace('\n', ' ')


In [ ]:
# this does not exists for all jobs, so handle the exceptions
salary_tag = card.find('span', 'salaryText')
if salary_tag:
    salary = salary_tag.text.strip()
else:
    salary = ''

In [ ]:
job_url = 'https://www.indeed.com' + card.h2.a.get('href')


In [ ]:
record = (job_title, company, job_location, post_date, today, summary, salary, job_url)


In [ ]:
record


Generalize the model with a function

In [ ]:

def get_record(card):
    """Extract job data from a single record"""

    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''

    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record

In [ ]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)


Get the next page

In [ ]:
while True:
    try:
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        break

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')

    for card in cards:
        record = get_record(card)
        records.append(record)

Putting it all together

In [ ]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup


def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url


def get_record(card):
    """Extract job data from a single record"""

    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''

    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)

    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break

    # save the job data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
        writer.writerows(records)


In [ ]:
# run the main program
main('senior accountant', 'charlotte nc')